In [93]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sbn
%matplotlib inline


def team_to_int_dict():
    teams = list(Game.objects.all().values_list('team', flat=True).distinct())
    return {team:teams.index(team) for team in teams}

team_ids = team_to_int_dict()

def prep_pass_attempts(player_list):
    return np.array([stat.pass_attempts for stat in player_list])

def prep_model_input(player_list):
    return np.array([[team_ids[stat.game.team],
                      stat.game.home] 
                      for stat in player_list])

def run_model(train_input, train_answer, test_input, test_answer):
    """Random Forest Classifier"""
    rfc = RandomForestClassifier ()
    rfc.fit(train_input, train_answer)
    predicted = rfc.predict(test_input)
    return accuracy_score(test_answer, predicted)

def compare_predicted_and_actual(test_answer, predicted):
    plt.figure(figsize=(13,7))
    plt.plot(test_answer)
    plt.plot(predicted)

train_input = prep_model_input(PlayerData.objects.filter(game__week__lte=13))
train_answer = prep_pass_attempts(PlayerData.objects.filter(game__week__lte=13))
test_input = prep_model_input(PlayerData.objects.filter(game__week=14))
test_answer = prep_pass_attempts(PlayerData.objects.filter(game__week=14))
run_model(train_input, train_answer, test_input, test_answer)

0.044247787610619468

In [ ]:
compare_predicted_and_actual